# Trabalho Prático - Word Embeddings
## Thiago Pádua de Carvalho - 2020007066

### Introdução
Neste trabalho prático, foi proposta a implementação de modelos de word embeddings e sua posterior avaliação em tarefas de similaridade e analogia de palavras. Para tal, foram criados modelos com variações de arquitetura e hiperparâmetros.
### Hiperparâmetros
Os hiperparâmetros utilizados foram:
- **Embedding Size**: Tamanho do vetor de embedding. Isto é, o número de dimensões do vetor de representação gerado para cada palavra no vocabulário. Por exemplo, para um embedding_size=2, cada palavra é representada por um vetor bidimensional. Como demonstração, [0.5, -0.3].
- **Window Size**: Tamanho da janela de contexto. Esse hiperparâmetro modifica como o cenário ao redor de uma palavra-alvo que será considerado durante o treinamento. Ele especifica quantas palavras à esquerda e à direita da referência o modelo deve se basear aprender suas representações.\
Por exemplo, se window_size=2, o modelo usa as 2 palavras imediatamente antes e as 2 imediatamente depois da palavra-alvo como contexto.
Exemplo: Para a frase "O modelo Word2Vec usa janelas de contexto", com window_size=2, o contexto da palavra "Word2Vec" seria ["O", "modelo", "usa", "janelas"].

- **Epochs**: Número de épocas de treinamento. Ele define a quantidade de vezes que o modelo percorre o corpus de treinamento completo durante o aprendizado, ou seja, quantas iterações sobre os dados serão feitas, afetando a convergência e a qualidade dos embeddings.

### Arquiteturas
Foram implementadas 2 arquiteturas de word embeddings:
- **Skip-gram**: Modelo que prevê as palavras do contexto a partir de uma palavra central.\
1 - Escolhe-se uma janela de contexto de tamanho k\
2 - O modelo é treinado para associar a palavra-alvo wt com cada palavra de contexto wc dentro da janela.\
3 - O objetivo do Skip-gram é maximizar a probabilidade conjunta de todas as palavras de contexto

- **CBOW**: Modelo que prevê a palavra central a partir das palavras do contexto:\
1 - Recebe as palavras de contexto (as palavras ao redor da palavra-alvo) como entrada.\
2 - Calcula a média dos vetores das palavras de contexto.\
3 - Tenta prever a palavra-alvo com base nessa média.

| **Aspecto**                  | **CBOW**                                      | **Skip-gram**                              |
|-------------------------------|-----------------------------------------------|--------------------------------------------|
| **Direção de predição**       | Contexto → Palavra-alvo                      | Palavra-alvo → Contexto                    |
| **Velocidade de treinamento** | Mais rápido                                  | Mais lento                                 |
| **Requisito de dados**        | Funciona bem com corpora pequenos            | Requer corpora maiores                    |
| **Foco semântico**            | Palavras frequentes                         | Palavras raras                            |
| **Robustez**                  | Melhor para contextos mais comuns           | Melhor para capturar relações profundas    |


### Implementação

In [1]:
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine

import itertools

In [2]:
with open("data/text8", "r") as file:
    corpus = file.read()

# Tokenize the corpus
sentences = [simple_preprocess(line) for line in corpus.splitlines()]

In [ ]:
def evaluate_model(model, analogy_file="data/questions-words.txt"):
    total_distance = 0
    num_analogies = 0

    with open(analogy_file, "r") as file:
        for line in file:
            if line.startswith(":"):  # Ignore categories
                continue

            words = line.strip().split()
            if all(word in model.wv for word in words):  # Check if all words are in the vocabulary
                vector_a = model.wv[words[1]]  # Example: France
                vector_b = model.wv[words[0]]  # Example: Paris
                vector_c = model.wv[words[2]]  # Example: Berlin
                vector_d = model.wv[words[3]]  # Example: Germany

                result_vector = vector_a - vector_b + vector_c

                # Compute the cosine distance between the resulting vector and the expected vector
                distance = cosine(result_vector, vector_d)
                total_distance += distance
                num_analogies += 1

    average_distance = total_distance / num_analogies
    return average_distance


In [ ]:
def generate_hyperparams_combinations(hyperparams):
    keys = hyperparams.keys()
    values = hyperparams.values()
    for combination in itertools.product(*values):
        yield dict(zip(keys, combination))

In [ ]:
hyperparams = {
    "embedding_size": [30, 50, 100, 200, 300, 500, 700],
    "window_size": [2, 3, 4, 5, 10],
    "skip_gram": [0, 1],
    "epochs": [5, 10, 15, 30, 40]
}

data_dir = "models"

results = []
for params in generate_hyperparams_combinations(hyperparams):
    model = Word2Vec(
        sentences=sentences,
        vector_size=params["embedding_size"],
        window=params["window_size"],
        sg=params["skip_gram"],
        epochs=params["epochs"]
    )

    model_name = f"word2vec_{params['embedding_size']}_{params['window_size']}_{params['skip_gram']}_{params['epochs']}.model"
    model.save(f"{data_dir}/{model_name}")

    avg_distance = evaluate_model(model)
    # print(f"Distância média: {avg_distance:.4f}")
    results.append((model_name, avg_distance, params))

In [ ]:
best_model = min(results, key=lambda x: x[1])
print(f"Best model: {best_model[0]} with average distance: {best_model[1]:.4f}")

Best model: word2vec_200_10_1_10.model with average distance: 0.8760
